In [110]:
import pandas as pd   # pip install pandas

df_faces = pd.read_excel('../stats/Преступность/11/ЛИЦА.xls', sheet_name='ЛИЦА-1', index_col=0) #считываем excel файл
#df_faces = df_faces.reset_index() #
df_faces = df_faces.rename(columns=lambda x: x if not 'Unnamed' in str(x) else None)
df_faces = df_faces.dropna()
df_faces.columns = pd.Series(df_faces.columns).fillna(method='ffill', axis=0)

df_faces

,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,...,Трудоспос. население в трудоспос. Возрасте,Трудоспос. население в трудоспос. Возрасте,Трудоспос. население в трудоспос. Возрасте,Трудоспос. население в трудоспос. Возрасте,Трудоспос. население в трудоспос. Возрасте,"Незанятые граждане,ищущие работу","Незанятые граждане,ищущие работу","Незанятые граждане,ищущие работу","Незанятые граждане,ищущие работу","Незанятые граждане,ищущие работу"
год,2010,2011,2012,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,...,2019,2018,2017,2016,2015,2019,2018,2017,2016,2015
Алейский,447,537,489,415.0,453.0,444.0,501.0,476.0,480.0,459.0,...,6316,6347,6860,7163,7483,155,141,143,135,167
Баевский,114,94,121,111.0,124.0,127.0,127.0,94.0,96.0,109.0,...,4065,4072,4439,4596,4866,243,193,182,197,181
Благовещенский,254,265,212,246.0,228.0,264.0,287.0,237.0,258.0,235.0,...,14444,14295,15037,15300,15867,406,293,353,405,387
Бурлинский,194,221,183,849.0,739.0,835.0,151.0,148.0,770.0,124.0,...,4945,4903,5207,5397,5620,276,236,257,266,301
Быстроистокский,134,124,92,144.0,101.0,121.0,112.0,92.0,105.0,84.0,...,3874,3795,4134,4238,4437,173,170,190,189,190
Егорьевский,178,165,172,174.0,192.0,198.0,181.0,128.0,133.0,184.0,...,6246,6284,6679,6850,7065,248,249,245,242,261
Ельцовский,96,104,95,112.0,116.0,93.0,81.0,155.0,81.0,80.0,...,2748,2740,2874,2934,3055,118,74,96,97,92
Завьяловский,189,206,239,255.0,243.0,250.0,233.0,169.0,176.0,178.0,...,8469,8362,9063,9246,9494,253,155,159,217,271
Залесовский,157,134,151,133.0,139.0,154.0,204.0,196.0,174.0,167.0,...,6773,6759,7042,7166,7407,297,230,238,286,299


In [111]:
df_faces.columns = [df_faces.columns, df_faces.iloc[0]]
df = df_faces.iloc[1:]
df.columns

MultiIndex([(                                     'ВСЕГО', 2010.0),
            (                                     'ВСЕГО', 2011.0),
            (                                     'ВСЕГО', 2012.0),
            (                                     'ВСЕГО', 2013.0),
            (                                     'ВСЕГО', 2014.0),
            (                                     'ВСЕГО', 2015.0),
            (                                     'ВСЕГО', 2016.0),
            (                                     'ВСЕГО', 2017.0),
            (                                     'ВСЕГО', 2018.0),
            (                                     'ВСЕГО', 2019.0),
            (                                     'ВСЕГО', 2020.0),
            (                'привлечено к уголовной отв', 2010.0),
            (                'привлечено к уголовной отв', 2011.0),
            (                'привлечено к уголовной отв', 2012.0),
            (                'привлечено к уголо

In [112]:
df_faces.iloc[1]

                                  год   
ВСЕГО                             2010.0    447.0
                                  2011.0    537.0
                                  2012.0    489.0
                                  2013.0    415.0
                                  2014.0    453.0
                                            ...  
Незанятые граждане,ищущие работу  2019.0    155.0
                                  2018.0    141.0
                                  2017.0    143.0
                                  2016.0    135.0
                                  2015.0    167.0
Name: Алейский, Length: 87, dtype: float64

In [113]:
dfcopy = df.copy()
dfcopy = dfcopy.loc[~dfcopy.index.duplicated(keep='first')]
dfcopy.stack().dropna().to_csv('outputForWeb.csv',header=True,index=True)
dfcopy = dfcopy.stack().dropna()
dfcopy = dfcopy.reset_index().rename(columns={'level_0': 'регион'})
dfcopy.columns= dfcopy.columns.str.lower()
dfcopy
dfcopy.to_csv('outputForWeb.csv',index=False)

In [115]:
dfcopy2 = dfcopy.sort_values("год", ascending=True).groupby(["год"]).transform('sum').drop_duplicates()
dfcopy_years = dfcopy[["год"]].drop_duplicates()
dfcopy_years
dfcopy2 = dfcopy2.reset_index(drop=True)
dfcopy2["год"] = dfcopy_years
dfcopy2 = dfcopy2.drop(['регион'], axis=1)
dfcopy2.to_csv('outputAll.csv',index=False)

In [116]:
dfcopy2.set_index('год', drop=True)

,всего,женщин,"незанятые граждане,ищущие работу",работников с/х,рабочих,служащих,трудоспос. население в трудоспос. возрасте,освобождено от уг отв,привлечено к уголовной отв
год,,,,,,,,,
2015.0,38961.0,5936.0,13148.0,310.0,7470.0,423.0,535830.0,16618.0,22343.0
2016.0,37682.0,5209.0,12413.0,293.0,6919.0,308.0,522066.0,14390.0,23060.0
2017.0,33959.0,4768.0,11566.0,180.0,6315.0,270.0,510257.0,11375.0,22511.0
2018.0,33717.0,4691.0,11203.0,204.0,6482.0,216.0,485848.0,11659.0,21977.0
2019.0,31194.0,4580.0,12649.0,126.0,6315.0,360.0,490675.0,10660.0,20411.0


In [14]:
def read_excel(data, path, sheet):
    data = pd.read_excel(path, sheet_name=sheet, index_col=0) #считываем excel файл
    data = data.reset_index() #
    data = data.rename(columns=lambda x: x if not 'Unnamed' in str(x) else None)
    #data = data.dropna()
    data.columns = pd.Series(data.columns).fillna(method='ffill', axis=0)
    return data

df_stats_all = pd.DataFrame()
df_stats_all = read_excel(df_stats_all, 'outputMain.xlsx', 'all')
df_stats_all_c = df_stats_all.copy()

In [15]:
df_faces.index = df_faces.index.str.lower()

In [16]:
df_faces_c = df_faces.copy()

In [17]:
df_faces_c = df_faces_c.loc[['год', 'итого'], 'ВСЕГО']
df_faces_c = df_faces_c.T[df_faces_c.T['год'] > 2013]
df_faces_c = df_faces_c[df_faces_c['год'] < 2019]
df_faces_c = df_faces_c.T
df_faces_c

KeyError: "None of [Index(['год', 'итого'], dtype='object')] are in the [index]"

In [ ]:
df_faces_c.columns = df_faces_c.iloc[0]
df_faces_c = df_faces_c.drop(df_faces_c.index[0])

In [ ]:
df_faces_c.T

In [ ]:
df_stats_all_c.loc[[6]]

In [ ]:
df_stats_all_c = df_stats_all.copy()
df_stats_all_c = df_stats_all_c.loc[[6]]
df_stats_all_c = df_stats_all_c.T
df_stats_all_c = df_stats_all_c.drop(df_stats_all_c.index[0])
df_stats_all_c

Проверка на нормальность

In [ ]:
from scipy import stats

[stats.shapiro(df_faces_c.squeeze()), stats.shapiro(df_stats_all_c.squeeze())]

Данные имеют нормальное распределение. Можно рассчитвать корреляцию Пирсона

In [ ]:
stats.pearsonr(df_faces_c.squeeze(), df_stats_all_c.squeeze())

Наблюдаем очень высокий коэффициент корреляции между значением преступности и уровнем безработицы в алтайском крае

In [ ]:
df_stud = read_excel(df_stats_all, 'outputStudMain.xlsx', 'Sheet1')
df_stud_c = df_stud.copy()
df_stud_c

In [ ]:
df_stud_c = df_stud_c.T
df_stud_c = df_stud_c.drop(df_stud_c.index[0])
df_stud_c

Проверка на нормальность

In [ ]:
[stats.shapiro(df_stud_c[0]), stats.shapiro(df_stud_c[1]), stats.shapiro(df_stud_c[2]), stats.shapiro(df_stud_c[3]),stats.shapiro(df_stud_c[4]),stats.shapiro(df_stud_c[5]),stats.shapiro(df_stud_c[6])] 

Видим, что у датафремов df_stud_c с индексами 1 и 3 распределение отлично от норамльного, т.к. pvalue < 0.05. Для них будем расчитывать корреляцию Кендалла

In [ ]:
df_stud_c[0]


In [ ]:
stats.rankdata(df_stats_all_c.squeeze())

Корреляция между уровнем безработицы и кол-вом обучающихся всего

In [ ]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[0])


Корреляция между уровнем безработицы и кол-вом человек в учреждениях СПО

In [ ]:
stats.kendalltau(stats.rankdata(df_stats_all_c.squeeze()), stats.rankdata(df_stud_c[1]))


Корреляция между уровнем безработицы и кол-вом принятых человек в учреждения СПО

In [ ]:
stats.spearmanr(df_stats_all_c.squeeze(), df_stud_c[2])


Корреляция между уровнем безработицы и кол-вом выпустников из учреждений СПО

In [ ]:
stats.kendalltau(stats.rankdata(df_stats_all_c.squeeze()), stats.rankdata(df_stud_c[3]))


Корреляция между уровнем безработицы и кол-вом человек в ВУЗах

In [ ]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[4])


Корреляция между уровнем безработицы и кол-вом человек принятых в ВУЗ

In [ ]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[5])


Корреляция между уровнем безработицы и кол-вом выпустников из ВУЗов

In [ ]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[6])

Высокая корреляция между уровнем безработицы и кол-вом выпустников из ВУЗов, следовательно при уменьшении кол-ва выпускников уровень безроботицы повышается. Но кол-во выпускников из учреждений СПО влияет на безработицу незначительно. Так же можем сказать, что при росте кол-ва обучающихся в целом, уровень безработицы уменьтшается